```{contents}
```
## Router Chain

A **Router Chain** is a LangChain abstraction that **dynamically routes an input to one of multiple downstream chains** based on the **content or intent of the input**.

> Instead of executing a fixed sequence, a router **decides which chain should handle the request**.

Conceptually, it introduces **conditional logic** into LLM pipelines.

---

### Why Router Chain Exists

Router chains solve problems where:

* Different inputs need different processing
* One prompt/model does not fit all tasks
* You want specialization without agents

Typical examples:

* Question answering vs summarization
* Technical vs non-technical queries
* SQL vs general knowledge queries
* Multiple domain-specific pipelines

---

### Core Idea

```
User Input
   ↓
Router (classification / decision)
   ↓
Selected Chain
   ↓
Final Output
```

Only **one downstream chain** is executed.

---

### Router Chain vs Sequential Chain

| Aspect      | SequentialChain | RouterChain |
| ----------- | --------------- | ----------- |
| Execution   | Always linear   | Conditional |
| Branches    | ❌               | ✅           |
| Parallelism | ❌               | ❌           |
| Determinism | High            | Medium      |

---

### How Routing Works Internally

1. Input is sent to a **router prompt**
2. Router decides a **destination name**
3. Matching chain is invoked
4. Output is returned

The router itself is usually **LLM-based**.

---

### Basic Router Chain Components

#### Router Prompt

Classifies or routes the input.

#### Destination Chains

Each handles a specific task.

#### Router Output Parser

Ensures valid destination selection.

---

### Basic Router Chain Demonstration

#### Step 1: Define Destination Chains



In [13]:
from langchain_classic.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

qa_prompt = PromptTemplate.from_template(
    "Answer the question clearly:\n{input}"
)

summary_prompt = PromptTemplate.from_template(
    "Summarize the following text:\n{input}"
)

qa_chain = LLMChain(llm=llm, prompt=qa_prompt)
summary_chain = LLMChain(llm=llm, prompt=summary_prompt)




---

#### Step 2: Define Router Prompt



In [14]:
from langchain_classic.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain_classic.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain_core.prompts import PromptTemplate

# Define destination information for the router
destinations = [
    {
        "name": "qa",
        "description": "Good for answering questions"
    },
    {
        "name": "summary", 
        "description": "Good for summarizing text"
    }
]

# Format destination strings
destinations_str = "\n".join([f"{d['name']}: {d['description']}" for d in destinations])

# Create the router prompt template
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

# Create the router chain
router_chain = LLMRouterChain.from_llm(
    llm=llm,
    prompt=router_prompt
)




The router prompt decides which destination is appropriate.

---

#### Step 3: Combine into a Router Chain




In [15]:
from langchain_classic.chains.router.multi_prompt import MultiPromptChain

destination_chains = {
    "qa": qa_chain,
    "summary": summary_chain,
}

multi_chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=qa_chain,
    verbose=True
)




---

#### Step 4: Invoke the Router Chain



In [16]:

multi_chain.invoke(
    {"input": "Summarize the LangChain framework"}
)




> Entering new MultiPromptChain chain...
summary: {'input': 'Summarize the LangChain framework.'}
> Finished chain.


{'input': 'Summarize the LangChain framework.',
 'text': 'The LangChain framework is designed to facilitate the development of applications that utilize language models. It provides a structured approach to integrate various components such as data sources, models, and output formats, enabling developers to create complex workflows. LangChain supports features like prompt management, memory handling, and chaining of multiple language model calls, making it easier to build sophisticated natural language processing applications. Its modular design allows for flexibility and scalability, catering to a wide range of use cases in AI-driven projects.'}



If input is a question → `qa_chain`
If input is long text → `summary_chain`

---

### How Routing Decision Looks (Conceptually)

Router output:

```json
{
  "destination": "summary",
  "next_inputs": {
    "input": "Summarize the LangChain framework"
  }
}
```

LangChain validates this before execution.

---

### Common Use Cases

* Intent-based routing
* Multi-domain chatbots
* Query type classification
* Specialized pipelines (RAG vs tools)
* Cost optimization (cheap vs expensive models)

---

### Router Chain vs Agents

| Aspect            | Router Chain | Agent     |
| ----------------- | ------------ | --------- |
| Decision          | One-time     | Iterative |
| Tool usage        | ❌            | ✅         |
| Complex reasoning | ❌            | ✅         |
| Predictability    | High         | Lower     |

Router chains are **simpler and cheaper** than agents.

---

### Router Chain Limitations

* ❌ Only one route selected
* ❌ No iterative reasoning
* ❌ No tool execution
* ❌ Legacy abstraction

---

### Router Chain vs LCEL Routing (Modern)

#### Router Chain (Legacy)

```python
MultiPromptChain(...)
```

### LCEL Conditional Routing (Recommended)

```python
def route(x):
    return qa_chain if "?" in x["input"] else summary_chain

chain = route | llm
```

LCEL provides more flexibility and composability.

---

### When to Use Router Chain

* Legacy LangChain codebases
* Simple intent routing
* Learning conditional workflows

---

### When NOT to Use Router Chain

* New projects
* Complex decision logic
* Tool-based reasoning
* Multi-step workflows

Use **LCEL** or **LangGraph** instead.

---

### Best Practices

* Keep router prompt simple
* Limit number of destinations
* Always define a default chain
* Log routing decisions
* Plan migration to LCEL

---

### Interview-Ready Summary

> “A Router Chain in LangChain dynamically selects one of multiple downstream chains based on input content. It enables conditional execution but is a legacy abstraction that has largely been replaced by LCEL and LangGraph for modern workflows.”

---

### Rule of Thumb

* **Simple conditional routing → Router Chain**
* **Modern pipelines → LCEL**
* **Dynamic reasoning → Agents or LangGraph**

